## Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

## csv to parquet
#### -> 메모리에 효율적인 데이터 유형을 사용하여 용량을 크게 줄이고 빠른 작업이 가능합니다.

In [2]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [4]:
csv_to_parquet('./train.csv', 'train')
csv_to_parquet('./test.csv', 'test')

train Done.
test Done.


## 데이터 불러오기

In [3]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

## 데이터 전처리

In [5]:
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [6]:
y_train = train['target'] 

X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 17)
(4701217,)
(291241, 17)


## 모델 선언 및 학습

In [7]:
LR = lgb.LGBMRegressor(random_state=42).fit(X_train, y_train)

## 추론

In [8]:
pred = LR.predict(test)

## Submission

In [9]:
sample_submission = pd.read_csv('./sample_submission.csv')

In [10]:
sample_submission['target'] = pred
sample_submission.to_csv("./submit.csv", index = False)

In [11]:
sample_submission

,id,target
0,TEST_000000,27.298710
1,TEST_000001,43.949243
2,TEST_000002,60.552621
3,TEST_000003,36.499322
4,TEST_000004,37.510570
...,...,...
291236,TEST_291236,45.902276
291237,TEST_291237,52.604831
291238,TEST_291238,21.964371
291239,TEST_291239,25.402628
